<a href="https://colab.research.google.com/github/jonathanahc/Proyecto/blob/main/ProyectoProgADAP_255497.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto de Programación para Analítica Descriptiva y Predictiva**
## Nombre del Proyecto: Consumo y costo del agua en Nueva York
## Grupo: A
## Autor: Jonathan Adrian Herrera Castro
## Matrícula: 255497
## Fecha: 19/02/2025

# **1. Introducción**

## - Descripción del problema: ¿Qué queremos analizar o predecir?
## - Objetivo del análisis: ¿Qué impacto tendrá este análisis?
## - Fuente de datos: ¿De dónde provienen los datos?


# **2. Captura y Carga de Datos**

In [32]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import re

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
df = pd.read_csv('/content/drive/MyDrive/ProyectoProgramacion/Water_Consumption_And_Cost__2013_-_Feb_2023__20250216.csv', engine="python")
df.head()

,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,...,Service End Date,# days,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (HCF),Water&Sewer Charges,Other Charges
0,HOWARD AVENUE,BROOKLYN,HOWARD AVENUE,BLD 02,AMR,NaN,339.0,782,K033900,FEDERAL,...,04/23/2020,31.0,E11310572,N,2945.22,Basic Water and Sewer,Yes,285,2945.22,0.0
1,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,...,01/26/2020,34.0,K13060723,N,196.35,Basic Water and Sewer,Yes,19,196.35,0.0
2,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,...,02/24/2020,29.0,K13060723,N,258.35,Basic Water and Sewer,Yes,25,258.35,0.0
3,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,...,03/23/2020,28.0,K13060723,N,217.02,Basic Water and Sewer,Yes,21,217.02,0.0
4,BAISLEY PARK,QUEENS,BAISLEY PARK,BLD 09,AMR,BLD 09,91.0,240,Q009100,FEDERAL,...,04/23/2020,31.0,K13060723,N,103.34,Basic Water and Sewer,Yes,10,103.34,0.0


In [34]:
print(f"Observaciones: {df.shape[0]}, Variables: {df.shape[1]}")

Observaciones: 50315, Variables: 25


#**3.Transformación y Preprocesamiento - Data Wrangling**

In [35]:
df.dtypes

,0
Development Name,object
Borough,object
Account Name,object
Location,object
Meter AMR,object
Meter Scope,object
TDS #,float64
EDP,int64
RC Code,object
Funding Source,object


In [36]:
df.isnull().sum()

,0
Development Name,60
Borough,0
Account Name,0
Location,828
Meter AMR,510
Meter Scope,37533
TDS #,60
EDP,0
RC Code,0
Funding Source,76


In [37]:
df.columns

Index(['Development Name', 'Borough', 'Account Name', 'Location', 'Meter AMR',
       'Meter Scope', 'TDS #', 'EDP', 'RC Code', 'Funding Source', 'AMP #',
       'Vendor Name', 'UMIS BILL ID', 'Revenue Month', 'Service Start Date',
       'Service End Date', '# days', 'Meter Number', 'Estimated',
       'Current Charges', 'Rate Class', 'Bill Analyzed', 'Consumption (HCF)',
       'Water&Sewer Charges', 'Other Charges'],
      dtype='object')

In [38]:
df.columns = df.columns.str.lower().str.replace(r"\W+", "", regex=True)

df.columns


Index(['developmentname', 'borough', 'accountname', 'location', 'meteramr',
       'meterscope', 'tds', 'edp', 'rccode', 'fundingsource', 'amp',
       'vendorname', 'umisbillid', 'revenuemonth', 'servicestartdate',
       'serviceenddate', 'days', 'meternumber', 'estimated', 'currentcharges',
       'rateclass', 'billanalyzed', 'consumptionhcf', 'watersewercharges',
       'othercharges'],
      dtype='object')

In [39]:
df['tds'] = df['tds'].fillna(0).astype(int)
df['days'] = df['days'].fillna(0).astype(int)
df['servicestartdate'] = pd.to_datetime(df['servicestartdate'], format='%m/%d/%Y')
df['serviceenddate'] = pd.to_datetime(df['serviceenddate'], format='%m/%d/%Y')
df['revenuemonth'] = pd.to_datetime(df['revenuemonth'], format='%Y-%m')

df.dtypes

,0
developmentname,object
borough,object
accountname,object
location,object
meteramr,object
meterscope,object
tds,int64
edp,int64
rccode,object
fundingsource,object


In [44]:
diccionario_datos = ({"Nombre de columna": df.columns,
                    "Tipo de dato": [str(df[col].dtype) for col in df.columns]})

print(diccionario_datos)

{'Nombre de columna': Index(['developmentname', 'borough', 'accountname', 'location', 'meteramr',
       'meterscope', 'tds', 'edp', 'rccode', 'fundingsource', 'amp',
       'vendorname', 'umisbillid', 'revenuemonth', 'servicestartdate',
       'serviceenddate', 'days', 'meternumber', 'estimated', 'currentcharges',
       'rateclass', 'billanalyzed', 'consumptionhcf', 'watersewercharges',
       'othercharges'],
      dtype='object'), 'Tipo de dato': ['object', 'object', 'object', 'object', 'object', 'object', 'int64', 'int64', 'object', 'object', 'object', 'object', 'int64', 'datetime64[ns]', 'datetime64[ns]', 'datetime64[ns]', 'int64', 'object', 'object', 'float64', 'object', 'object', 'int64', 'float64', 'float64']}


# **4. Análisis Descriptivo de Datos**

## - Preguntas a responder con el AD
## - Explicación de Estadísticas Básicas Aplicadas, Visualización de Datos con Gráficos, y Resumen de las respuestas

# **5. Análisis Exploratorio de los Datos**

## - Preguntas a responder
## - Estadísticas Empleadas, visualización y Resumen de las respuestas encontradas.

# **6. Análisis Predictivo**

## - Preprocesamiento para Análisis Predictivo: Problemas, Técnicas y Código

## - Modelado Predictivo
### - Modelo elegido y parámetros
### - Entrenamiento y Evaluación
## - Resultados


# **7. Conclusiones y Trabajos Futuros**

## - Resumen del trabajo realizado, hallazgos y posibles mejoras